#### Importation

In [3]:
from ultralytics import YOLO
import glob
import cv2
import os


In [4]:
# Créer le modèle YOLOv8 à partir d'un modèle pré-entraîné
model = YOLO("yolov8n.pt")

In [ ]:


# Lancer l'entraînement
model.train(
    data="tomato_yolo/data.yaml",
    epochs=3,
    batch=16,
    imgsz=640,
    name="tomato_yolo_model" 
)


New https://pypi.org/project/ultralytics/8.3.221 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.220  Python-3.13.9 torch-2.9.0+cpu CPU (Intel Core i5-7200U 2.50GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tomato_yolo/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=tomato_yolo_model6, nbs=64, nms=False, opset

KeyboardInterrupt: 

In [5]:
# Évaluer sur le dataset de validation défini dans data.yaml
metrics = model.val(data="tomato_yolo/data.yaml", batch=16, imgsz=640)

# Les métriques principales sont dans metrics.boxes
print("📊 Métriques du modèle :")
print(metrics)

Ultralytics 8.3.220  Python-3.13.9 torch-2.9.0+cpu CPU (Intel Core i5-7200U 2.50GHz)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access  (ping: 0.50.2 ms, read: 1.90.5 MB/s, size: 34.7 KB)
val: Scanning C:\Computer Vision\Jour 5\tomato_yolo\valid\labels.cache... 61 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 61/61 19.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.2it/s 22.3s8.5ss
                   all         61        196     0.0084      0.015    0.00722    0.00554
                person          3          4          0          0          0          0
               bicycle         31         96          0          0          0          0
                   car         19         19     0.0588      0.105     0.0505     0.0388
            motorcycle         13         29          0          0          0          0
              airplane         

In [6]:
# Charger le modèle entraîné
model = YOLO("runs/detect/tomato_yolo_model4/weights/best.pt")

In [7]:
# Parcourir toutes les images d’un dossier
images = glob.glob("tomato_yolo/test/images/*.jpg")

In [8]:
# prendre la première image
img_path = images[0]

In [10]:
# Autre image
img_path = images[19]

In [12]:
# Autre image
img_path = images[29]

In [13]:
# Lire l'image
img = cv2.imread(img_path)

# Faire la prédiction
results = model.predict(img, conf=0.25)[0]

# Dessiner les boîtes et afficher le nom de la maladie
for box, cls, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
    x1, y1, x2, y2 = map(int, box)
    label = model.names[int(cls)]       # Nom de la maladie
    confidence = float(conf)            # Confiance

    # Dessiner la boîte verte
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Afficher le label sur l’image
    cv2.putText(img, f"{label} {confidence:.2f}", (x1, y1-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# Afficher l’image annotée
cv2.imshow("Maladie détectée", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# sauvegarder l'image annotée
output_folder = "outputs"
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, os.path.basename(img_path))
cv2.imwrite(output_path, img)
print(f"Image annotée sauvegardée dans : {output_path}")


0: 640x640 3 Leaf_Molds, 528.6ms
Speed: 16.1ms preprocess, 528.6ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)
Image annotée sauvegardée dans : outputs\IMG_1141_JPG.rf.974b08fa79db2a3fe741383ba39e8603.jpg
